In [ ]:
##################################
## Importing necessary packages ##
##################################

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
from scipy.stats import multivariate_normal
import pandas as pd
from itertools import combinations
from sklearn.neighbors import KernelDensity
import random
import copy
import rasterio
import itertools
import time
import mpi4py
from mpi4py import MPI
import os
import math

##################################
## Define output folder pathway ##
##################################

PATH = './PATH_FILE’

##################################################################
## Designate each model repetition with a separate ‘rank’ value ##
##################################################################

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

#####################################################
## Create output subfolders for each output metric ##
#####################################################

if rank == 0:
    os.mkdir(PATH)
    os.mkdir(PATH+"/Total_deer")
    os.mkdir(PATH+"/Total_exp_deer")    
    os.mkdir(PATH+"/Total_infect_deer")        
    os.mkdir(PATH+"/Prevalence")
    os.mkdir(PATH+"/Direct_trans")
    os.mkdir(PATH+"/Indirect_trans")    
    os.mkdir(PATH+"/Max_life")        
    os.mkdir(PATH+"/Max_disease")
    os.mkdir(PATH+"/Disease_mort")
    os.mkdir(PATH+"/Emigrants")
    os.mkdir(PATH+"/Deer_culled")
    os.mkdir(PATH+"/Prion_cells")

#################################################################
## Specify basemaps by directing to a map configuration folder ##
#################################################################

maps = np.loadtxt('./config_twp',dtype=str)

#####################################
## Specify variable values (daily) ##
#####################################

DIR_TRANS = 0.00065 
MALE_ADDED_DT = 6
DIS_MORT = 0.00015 
GROUPS = 251  
IND_TRANS = 0.00029 
HALF_LIFE = 48

decay_rate = (-np.log(0.5)/HALF_LIFE) 

BIRTH_RATE = 0.0167

ADULT_MALE_HARVEST_RATE = 0.0025
ADULT_FEMALE_HARVEST_RATE = 0.001
YEARLING_MALE_HARVEST_RATE = 0.003
YEARLING_FEMALE_HARVEST_RATE = 0.00144
ADULT_MALE_BASE_MORT_RATE = 0.001
ADULT_FEMALE_BASE_MORT_RATE = 0.00033
YEARLING_MALE_BASE_MORT_RATE = 0.0023
YEARLING_FEMALE_BASE_MORT_RATE = 0.0025
FAWN_TWO_MONTH_MORT_RATE = 0.0055
FAWN_TEN_MONTH_MORT_RATE = 0.00026

#Number of young males leaving and entering the study area
YEARLING_FALL_EM_RATE = 0.00267
YEARLING_SPRING_EM_RATE = 0.00267
ADULT_FALL_EM_RATE = 0
ADULT_SPRING_EM_RATE = 0

#Number of young males changing groups within study area
YEARLING_FALL_DISP_RATE = 0.00433
YEARLING_SPRING_DISP_RATE = 0.01267
ADULT_FALL_DISP_RATE = 0
ADULT_SPRING_DISP_RATE = 0 

###########################################################
## Specify map details for underlying layers (e.g., RSF) ##
###########################################################

bm = rasterio.open(maps[0])

bm_rows = bm.height #rows
bm_cols = bm.width #cols
bm_cs = bm.res[0] #cell size
bm_xllc = bm.bounds[0] #xllcorner
bm_yllc = bm.bounds[1] #yllcorner

BINARY_MAP = bm.read()[0,:,:]
BINARY_MAP[BINARY_MAP == -9999] = 0

rm = rasterio.open(maps[1])

rm_rows = rm.height #rows
rm_cols = rm.width #cols
rm_cs = rm.res[0] #cell size
rm_xllc = rm.bounds[0] #xllcorner
rm_yllc = rm.bounds[1] #yllcorner

RSF_MAP = rm.read()[0,:,:]
RSF_MAP[RSF_MAP == -9999] = 0

rem = rasterio.open(maps[2]) 

REMOVE_MAP=rem.read()[0,:,:]
REMOVE_MAP[REMOVE_MAP == -9999] = 0

rem_rows = rem.height
rem_cols = rem.width
rem_cs = rem.res[0]
rem_xllc = rem.bounds[0]
rem_yllc = rem.bounds[1]

#################################################
## Defining a distance function between 2 deer ##
#################################################

def dist(p1, p2):
    (x1, y1), (x2, y2) = p1, p2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

##################################################
## Initializes a world for the deer to exist on ##
##################################################

class World():
    def __init__(self,total_timesteps,suscept_yrling_males=0,infected_yrling_males=0,suscept_yrling_females=0,infected_yrling_females=0,suscept_adult_males=0,infected_adult_males=0,suscept_adult_females=0,infected_adult_females=0,suscept_fawn_males=0,infected_fawn_males=0,suscept_fawn_females=0,infected_fawn_females=0,groups=0,params='',xtrds='',ytrds='',cov_mats=''):

        self.y_size = bm_rows
        self.x_size = bm_cols

        self.days = 0 

        self.dir_transmissions = 0
        self.ind_transmissions = 0
        self.max_life = 0
        self.max_disease = 0
        self.dz_mort = 0
        self.emigrants = 0
        self.groups = groups 

        self.culled_deer = 0 
        self.infectious_culled = False
        self.infectious_harvested = False
        
        self.prion_deposited_cells=0
        self.prion_map = np.zeros((self.y_size,self.x_size))
        self.deer_list = []
        self.male_list = []  
        self.female_list = []  
        self.herd_list = []
        self.cov_mats = cov_mats
        self.params = params
        
        self.group_pos = [(681636,4742528)]
        self.dz_detected= False 

        while len(self.group_pos) < groups:
            possible_locs = []
            possible_loc_vals = []
            for n in range(3):
                rand_loc = np.random.randint(len(np.where(BINARY_MAP==1)[0]))  
 
                pos_x,pos_y = np.where(BINARY_MAP==1)[1][rand_loc], np.where(BINARY_MAP==1)[0][rand_loc]  

                possible_locs.append((pos_x,pos_y))

                possible_loc_vals.append(RSF_MAP[pos_y,pos_x])

            x_loc = rm_xllc + possible_locs[np.argmax(possible_loc_vals)][0]*rm_cs
            y_loc = rm_yllc + possible_locs[np.argmax(possible_loc_vals)][1]*rm_cs

            distances_from_groups = [np.inf]
            for other in self.group_pos:
                distances_from_groups.append(dist((x_loc,y_loc),other))

            if np.min(distances_from_groups) > 150:
                self.group_pos.append((x_loc,y_loc))

       
        for h in range(groups):

            loc = np.random.randint(params.shape[0])
            self.herd_list.append(Herd(self.group_pos[h],params[loc,:],xtrds[loc],ytrds[loc],total_timesteps))

        for m in range(suscept_yrling_males):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            # deer start at herd center with a bit of noise
           
self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=False,age=np.random.randint(17,21),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g)) #if begin trial in Jan
        for f in range(suscept_yrling_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=False,age=np.random.randint(17,21),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for m in range(infected_yrling_males):
            loc = np.random.randint(params.shape[0])
            g =0 # Puts the infected deer in the center of the world
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=True,age=np.random.randint(17,21),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for f in range(infected_yrling_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=True,age=np.random.randint(17,21),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for m in range(suscept_adult_males):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=False,age=np.random.randint(29,65),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for f in range(suscept_adult_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=False,age=np.random.randint(29,65),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for m in range(infected_adult_males):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=True,age=np.random.randint(29,65),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for f in range(infected_adult_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=True,age=np.random.randint(29,65),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for m in range(suscept_fawn_males):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=False,age=np.random.randint(5,8),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for f in range(suscept_fawn_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=False,age=np.random.randint(5,8),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for m in range(infected_fawn_males):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Male',inf=False,exp=True,age=np.random.randint(5,8),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))
        for f in range(infected_fawn_females):
            loc = np.random.randint(params.shape[0])
            g = np.random.randint(0,groups)
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1),gender='Female',inf=False,exp=True,age=np.random.randint(5,8),params=params[loc,:],cov_mat=cov_mats[loc,:,:],herd=g))

##################################
## World directs how deer move ##
##################################

    def move_deer(self):

        b_hops = []
        for h in self.herd_list:
            b_hops.append(h.basin_hop(self.days))

        for d in self.deer_list: 
d.move(b_hops[d.herd_id],self.herd_list[d.herd_id].cx,self.herd_list[d.herd_id].cy)

    def initialize_infection(self):
        group_zero = [d for d in self.deer_list if d.herd_id ==0]
        np.random.choice(group_zero).infectious=True

    def fall_dispersal(self):
        for d in self.deer_list:
            if d.gender == 'Male' and 12<=d.age<24 and d.disp_int >= 5: 
                if np.random.uniform(0,1) < YEARLING_FALL_DISP_RATE:
                    tot_groups = list(range(len(self.herd_list)))
                    tot_groups.remove(d.herd_id) 

                    new_group = np.random.choice(tot_groups)

                    d.herd_id = new_group
                    d.disp_int = 0
                    
            if d.gender == 'Male' and 24<=d.age<36 and d.disp_int>=5:
                if np.random.uniform(0,1) < 0.003:
                    tot_groups = list(range(len(self.herd_list)))
                    tot_groups.remove(d.herd_id)

                    new_group = np.random.choice(tot_groups)

                    d.herd_id = new_group
                    d.disp_int = 0
                    
    def spring_dispersal(self):
        for d in self.deer_list:
            if d.gender == 'Male' and 12<=d.age<24 and d.disp_int >= 5:
                if np.random.uniform(0,1) < YEARLING_SPRING_DISP_RATE:
                    tot_groups = list(range(len(self.herd_list)))
                    tot_groups.remove(d.herd_id)

                    new_group = np.random.choice(tot_groups)

                    d.herd_id = new_group
                    d.disp_int = 0
             
    def spring_emigration(self):
        loc=np.random.randint(self.params.shape[0])
        g=np.random.randint(0,self.groups)
        for d in self.deer_list:
            if d.gender == 'Male' and 11<=d.age<=22:
                if np.random.uniform(0,1) < YEARLING_SPRING_EM_RATE:
                    d.alive = False   
                    if d.exposed == True or d.infectious == True:
                        self.emigrants += 1

    def spring_immigration(self):
        loc=np.random.randint(self.params.shape[0])
        g=np.random.randint(0,self.groups)
        if np.random.uniform(0,1) < YEARLING_SPRING_EM_RATE:
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1), gender='Male',inf=False,exp=False, age=np.random.randint(11,13),params=self.params[loc,:],cov_mat=self.cov_mats[loc,:,:],herd=g))
                        
    def fall_emigration(self):
        loc=np.random.randint(self.params.shape[0])
        g=np.random.randint(0,self.groups)
        for d in self.deer_list:
            if d.gender == 'Male' and 14 < d.age <= 18:
                if np.random.uniform(0,1) < YEARLING_FALL_EM_RATE:
                    d.alive = False   
                    if d.exposed == True or d.infectious == True:
                        self.emigrants += 1
                            
    def fall_immigration(self):
        loc=np.random.randint(self.params.shape[0])
        g=np.random.randint(0,self.groups)
        if np.random.uniform(0,1) < YEARLING_FALL_EM_RATE:
            self.deer_list.append(Deer(x=self.group_pos[g][0]+np.random.normal(10,1),y=self.group_pos[g][1]+np.random.normal(10,1), gender='Male',inf=False,exp=False, age=np.random.randint(14,18),params=self.params[loc,:],cov_mat=self.cov_mats[loc,:,:],herd=g))                     
                    
    def incr_age(self):
        for d in self.deer_list:
            d.age_deer()

            if d.age >= 216 and d.alive:
                d.alive = False
                self.max_life += 1

    def incr_time_inf(self):
        for d in self.deer_list:
            d.infected_count()

    def incr_birth_int(self):
        for d in self.deer_list:
            d.birth_interval()

    def incr_disp_int(self):
        for d in self.deer_list:
            d.disp_interval()

    def birth(self):
        for d in self.deer_list:
            if d.age>=22 and d.gender=='Female' and d.birth_int>=5:
            #Adults have a mean litter size of 2.0 
                if BIRTH_RATE > np.random.uniform():
                    litter_size = np.random.choice([0,1,2,3], p=[0.070747,0.158505,0.670748,0.1])
                    d.birth_int = 0
                    for f in range(litter_size):
                        loc = np.random.randint(self.params.shape[0])
                        self.deer_list.append(Deer(x=d.x, y=d.y, inf=False, exp=False, gender=random.choice(['Male','Female']), age=0,params=self.params [loc,:],cov_mat=self.cov_mats[loc,:,:],herd=d.herd_id))

            if 13<=d.age<22 and d.gender=='Female' and d.birth_int>=5:
            #Yearlings have mean of 1.8 fawns/litter
                if BIRTH_RATE > np.random.uniform():
                    litter_size = np.random.choice([0,1,2,3], p=[0.05,0.45,0.45,0.05])
                    d.birth_int = 0
                    for f in range(litter_size):
                        loc = np.random.randint(self.params.shape[0])
                        self.deer_list.append(Deer(x=d.x, y=d.y, inf=False, exp=False, gender=random.choice(['Male','Female']), age=0,params=self.params [loc,:],cov_mat=self.cov_mats[loc,:,:],herd=d.herd_id))

            if d.age<13 and d.gender=='Female' and d.birth_int>=5:
            #Fawns have mean of 1.2 fawns
                if BIRTH_RATE > np.random.uniform():
                    litter_size = np.random.choice([0,1,2,3],p=[0.5,0.5,0,0])
                    d.birth_int = 0
                    for f in range(litter_size):
                        loc = np.random.randint(self.params.shape[0])
                        self.deer_list.append(Deer(x=d.x, y=d.y, inf=False, exp=False, gender=random.choice(['Male','Female']), age=0,params=self.params [loc,:],cov_mat=self.cov_mats[loc,:,:],herd=d.herd_id))


    def shed_prions(self): # Greater deposition amount for deer with more DPI
        for d in self.deer_list:
            if d.infectious == True and d.alive == True:
                self.prion_map[int((d.y-rem_yllc)/rem_cs),int((d.x-rem_xllc)/ rem_cs)] += 1*(int(d.time_infected/30)+1)  

            if d.infectious == True and d.alive == False:   
                self.prion_map[int((d.y-rem_yllc)/rem_cs),int((d.x-rem_xllc)/ rem_cs)]+=50*(int(d.time_infected/30)+1) 

    def prion_decay(self):
        self.prion_map *= 1-decay_rate

    def ind_trans(self):
        for d in self.deer_list:
            if d.infectious == False and d.exposed == False:
                if d.gender=='Male':
                    if np.random.uniform() < 1 - np.exp(-(IND_TRANS*2)* self.prion_map[int((d.y-bm_yllc)/bm_cs),int((d.x-bm_xllc)/bm_cs)]/bm_cs**2):
                        d.exposed = True
                        self.ind_transmissions += 1
                if d.gender=='Female':
                    if np.random.uniform() < 1 - np.exp(-IND_TRANS* self.prion_map[int((d.y-bm_yllc)/bm_cs),int((d.x-bm_xllc)/bm_cs)]/bm_cs**2):
                        d.exposed = True
                        self.ind_transmissions += 1
    def dir_trans_jan(self,dir_radius):            
        '''
        January direct transmission events
        For every pair of deer where one is infectious and one is susceptible 
        '''
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1        
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*2 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1        
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*2 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                            
    def dir_trans_feb(self,dir_radius):
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1      
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                                          
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*3 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1      
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                                          
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*3 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1

    def dir_trans_mar(self,dir_radius):
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                                                  
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*3 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                                                  
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*3 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*7 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                

    def dir_trans_apr(self,dir_radius):
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                          
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':        
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                          
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                    
    def dir_trans_may(self,dir_radius):
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                          
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*2 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                          
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                              
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*2 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                    
    def dir_trans_june(self,dir_radius):
        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                            
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                            
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                            
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    
    def dir_trans_july(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False: 
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                    
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1       
                            
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*3 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                    
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1

    def dir_trans_aug(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*7 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                        
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if DIR_TRANS*7 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1  
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                        
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                    
    def dir_trans_sept(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*7 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                    
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*7 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                    
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                            
    def dir_trans_oct(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1    
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*5 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*5 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1    
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*5 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if MALE_ADDED_DT*DIR_TRANS*5 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 
                        
    def dir_trans_nov(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1        
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                       
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Male' and deer_2.gender=='Female':
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1        
                    if deer_1.gender=='Female' and deer_2.gender=='Female':
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                       
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 
                        
    def dir_trans_dec(self,dir_radius):

        for deer_1,deer_2 in itertools.combinations(self.deer_list,2):
            if (deer_1.infectious ^ deer_2.infectious) and (deer_1 - deer_2 < dir_radius):
                if deer_1.infectious==True and deer_2.exposed==False:
                    if deer_1.gender=='Male' and deer_2.gender=='Male': 
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male': 
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1
                                
                if deer_1.infectious==False and deer_2.exposed==True:
                    if deer_1.gender=='Male' and deer_2.gender=='Male': 
                        if MALE_ADDED_DT*DIR_TRANS*1 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1 
                    if deer_1.gender=='Male' and deer_2.gender=='Female': 
                        if DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1
                    if deer_1.gender=='Female' and deer_2.gender=='Male':                         
                        if MALE_ADDED_DT*DIR_TRANS*2 > np.random.uniform():
                            deer_2.exposed=True
                            self.dir_transmissions += 1                
                    if deer_1.gender=='Female' and deer_2.gender=='Female': 
                        if deer_1.herd_id != deer_2.herd_id:  
                            if DIR_TRANS*1 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1                            
                        if deer_1.herd_id == deer_2.herd_id:
                            if DIR_TRANS*4 > np.random.uniform():
                                deer_2.exposed=True
                                self.dir_transmissions += 1 

    def disease_mort(self):
        '''
        This function simulates CWD-associated mortality
        Sets infected deer to dead after 5 years post-initial infection. Daily probabilities
        '''
        for d in self.deer_list:
            if d.time_infected > 300 and np.random.uniform() < DIS_MORT: 
                d.alive = False      
                self.dz_mort += 1
            if d.time_infected >= 1800: 
                d.alive = False
                self.max_disease += 1
                
    def harvest_mort(self):
        '''
        This function simulates harvest mortality for each deer demographic
        '''
        for d in self.deer_list:
            if d.gender=='Female' and 11 < d.age < 23 and np.random.uniform() < YEARLING_FEMALE_HARVEST_RATE:
                d.alive = False                
            if d.gender=='Male' and 11 < d.age < 23 and np.random.uniform() < YEARLING_MALE_HARVEST_RATE: 
                d.alive = False
            if d.gender=='Male' and d.age >=23 and np.random.uniform() < ADULT_MALE_HARVEST_RATE:
                d.alive = False
            if d.gender=='Female' and d.age >=23 and np.random.uniform() < ADULT_FEMALE_HARVEST_RATE:
                d.alive = False

    def fawn_mort(self):
        '''
        This function simulates increased mortality of fawns during their first 12 months of life (predation, abandonment, etc.)
        Fawn harvest mortality included in this parameter
        '''
        for d in self.deer_list:
            if 0<=d.age<2 and np.random.uniform()<FAWN_TWO_MONTH_MORT_RATE:
                d.alive = False
            if 2<=d.age<=12 and np.random.uniform()<FAWN_TEN_MONTH_MORT_RATE:
                d.alive = False

    def baseline_mort(self):
        '''
        This function simulates baseline mortality of deer during the non-hunting season and to keep population stationary (i.e., not growing; predation, DVCs, etc.)
        '''
        for d in self.deer_list:
            if d.gender == 'Male' and d.age >= 23 and np.random.uniform() < ADULT_MALE_BASE_MORT_RATE:
                d.alive = False
            if d.gender == 'Female' and d.age >= 23 and np.random.uniform() < ADULT_FEMALE_BASE_MORT_RATE:
                d.alive = False
            if d.gender == 'Male' and 11 < d.age < 23 and np.random.uniform() < YEARLING_MALE_BASE_MORT_RATE:
                d.alive = False
            if d.gender == 'Female' and 11 < d.age < 23 and np.random.uniform() < YEARLING_FEMALE_BASE_MORT_RATE:
                d.alive = False

    def male_comp(self):
        self.male_list = [d for d in self.deer_list if d.alive==True and d.gender=='Male']

    def female_comp(self):
        self.female_list = [d for d in self.deer_list if d.alive==True and d.gender=='Female']

    def list_comp(self):
        self.deer_list = [d for d in self.deer_list if d.alive==True]

    def increment_day(self):
        self.days += 1

#############################################
## Calculate statistics for each time step ##
#############################################

    def stats(self):
        total_deer = len(self.deer_list)
        exposed_deer = 0
        infectious_deer = 0

        total_males = len(self.male_list)
        exposed_males = 0
        infectious_males = 0

        total_females = len(self.female_list)
        exposed_females = 0
        infectious_females = 0
        
        total_cells = int(rm_rows*rm_cols)
        prion_deposited_cells = (self.prion_map!=0).sum()
        
        for d in self.deer_list:
           if d.infectious == True:
               infectious_deer += 1
           elif d.exposed == True:
               exposed_deer += 1
           else:
               pass

        for d in self.male_list:
            if d.infectious == True:
                infectious_males += 1
            elif d.exposed == True:
                exposed_males += 1
            else:
                pass

        for d in self.female_list:
            if d.infectious == True:
                infectious_females += 1
            elif d.exposed == True:
                exposed_females += 1
            else:
                pass

        return total_deer, exposed_deer, infectious_deer, ((exposed_deer+infectious_deer)/total_deer),self.dir_transmissions, self.ind_transmissions, self.max_life, self.max_disease, self.dz_mort, self.emigrants, ((exposed_males+infectious_males)/ total_males), ((exposed_females+infectious_females)/total_females),(prion_deposited_cells/total_cells)


###################################################################
## Initializes the agents (deer) and functions happening to them ##
###################################################################

class Deer():
    def __init__(self,x=None,y=None,inf=False,exp=False,gender='Male',age=0,time_infected=0,birth_int=0,disp_int=0,params='',cov_mat='',herd=0):
        self.x = x
        self.y = y

        self.infectious = inf
        self.exposed = exp
        self.gender = gender
        self.alive = True
        self.age = age
        self.time_infected = time_infected
        self.birth_int = 0
        self.disp_int = 0
        self.cov_mat = cov_mat
        self.params = params
        self.herd_id = herd

    def move(self,b_hop,cx,cy):

        has_moved = False
        
        attempts = 0
        while has_moved == False and attempts < 100:
            rand = self.rng(1)

            new_x=self.x-self.params[0]*(self.x-cx) + rand[0][0] + b_hop[0]
            new_y=self.y-self.params[1]*(self.y-cy) + rand[1][0] + b_hop[1]
            
            # Check on board
            if int((new_y-bm_yllc)/bm_cs) < bm_rows and int((new_y-bm_yllc)/ bm_cs) >= 0 and int((new_x-bm_xllc)/bm_cs) < bm_cols and int((new_x-bm_xllc)/ bm_cs) >= 0:
            
                # CHECK BINARY MAP
                if BINARY_MAP[int((new_y - bm_yllc)/bm_cs), int((new_x - bm_xllc)/bm_cs)]:
                    # stationary part
                    self.x = new_x
                    self.y = new_y
                    has_moved = True
                
                else:
                    attempts += 1
            else:
                attempts += 1

    def rng(self,n):
        mvnorm = stats.multivariate_normal(mean=[0,0], cov=self.cov_mat)

        x = mvnorm.rvs(n).reshape(n,2)

        # Create univariate normal dists to invert marginals
        norm1 = stats.norm(0,np.sqrt(self.cov_mat[0,0]))
        norm2 = stats.norm(0,np.sqrt(self.cov_mat[1,1]))

        # Transform the Gaussian samples to a uniform distribution
        x_unif1 = norm1.cdf(x[:,0])
        x_unif2 = norm2.cdf(x[:, 1])

        # Use ML estimator for Laplace dist
        bx = self.params[5]
        by = self.params[6]

        # Construct the Laplace dist
        m1 = stats.laplace(0,bx)
        m2 = stats.laplace(0,by)

        # Transform the Uniform dist to a Laplace dist
        x1_trans = m1.ppf(x_unif1)
        x2_trans = m2.ppf(x_unif2)

        return x1_trans,x2_trans

    def age_deer(self):
        self.age += 1

    def infected_count(self):
        if self.exposed or self.infectious:
            self.time_infected += 1

        if self.time_infected > np.random.randint(90,180): 
            self.infectious = True
            self.exposed = False

    def birth_interval(self):
        self.birth_int += 1

    def disp_interval(self):
        self.disp_int += 1
        
    def __sub__(self, other):
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

#################################################################
## Initializes the deer groups and functions happening to them ##
#################################################################

class Herd():
    def __init__(self,pos,params,xtrds,ytrds,total_timesteps):

        self.params = params
        self.trd_x = xtrds
        self.trd_y = ytrds
        self.kde = self.make_kde()

        self.cx = pos[0]
        self.cy = pos[1]

        self.lasso_times = self.lasso_times(total_timesteps)

    def make_kde(self):

        maxCol=lambda x: max(x.min(), x.max(), key=abs)

        smth_lasso_jumps = np.stack([pd.DataFrame(np.diff(self.trd_x)).rolling(3,center=True).apply(maxCol).dropna().to_numpy().flatten(),pd.DataFrame(np.diff(self.trd_y)).rolling(3,center=True).apply(maxCol).dropna().to_numpy().flatten()],axis=1)

        pts = smth_lasso_jumps

        points = list(zip(np.diff(pts[:,0]),np.diff(pts[:,1])))
        distances = [dist(p1, p2) for p1, p2 in combinations(points, 2)]
        avg_distance = sum(distances) / len(distances)

        lasso_pdf = KernelDensity(kernel='gaussian',bandwidth=avg_distance*.5)
        lasso_pdf.fit(smth_lasso_jumps)

        return lasso_pdf
    
    
    def basin_hop(self,time):

        if np.any(self.lasso_times == time):

            jump = self.kde.sample(3)

            possible_jumps = []
            attempts = 0
            while len(possible_jumps) < 3: # CHANGED THIS TO ADD < 3:
                new_x,new_y = jump[len(possible_jumps)] + [self.cx,self.cy]
                
                if int((new_y - rm_yllc)/rm_cs) < rm_rows and int((new_y - rm_yllc)/rm_cs) >= 0 and int((new_x - rm_xllc)/rm_cs) < rm_cols and int((new_x - rm_xllc)/rm_cs) >= 0:
                    
                    possible_jumps.append(RSF_MAP[int((new_y-rm_yllc)/rm_cs),int((new_x-rm_xllc)/rm_cs)])
                
                else:
                    attempts += 1
                
                if attempts > 100:
                    return [0,0]

            x_jump_ = jump[np.argmax(possible_jumps)][0]
            y_jump_ = jump[np.argmax(possible_jumps)][1]

            # move the center
            self.cx += x_jump_
            self.cy += y_jump_

            return [x_jump_,y_jump_]

        else:
            return [0,0]

    def lasso_times(self,total_timesteps):

        num_crit_jumps_data = self.params[3]

        jump_times = [np.random.exponential(self.params[4])]
        while np.cumsum(jump_times)[-1] <= total_timesteps:
            jump_times.append(np.random.exponential(self.params[4]))
        return jump_times

fits = np.load('./Data/fits.npy')
xtrends = np.load('./Data/xtrends.npy',allow_pickle=True)
ytrends = np.load('./Data/ytrends.npy',allow_pickle=True)
cov_matrices = np.load('./Data/covar_matrices.npy')


##################################
## Initializes Simulation Trial ##
##################################

def disease(timesteps):
    np.random.seed()

    data = np.zeros((13,timesteps))
    
    dir_radius = 100 
    
    T_total_deer=np.zeros(timesteps)
    T_exposed_deer=np.zeros(timesteps)
    T_infectious_deer=np.zeros(timesteps)
    T_prevalence=np.zeros(timesteps)
    T_dir_trans=np.zeros(timesteps)
    T_ind_trans=np.zeros(timesteps)
    T_max_life=np.zeros(timesteps)
    T_max_disease=np.zeros(timesteps)
    T_dz_mort=np.zeros(timesteps)
    T_emigration=np.zeros(timesteps)
    T_m_prevalence=np.zeros(timesteps)
    T_f_prevalence=np.zeros(timesteps)       
    T_shed_prions=np.zeros(timesteps)  
       
    trial = World(total_timesteps=timesteps,suscept_yrling_males=111,infected_yrling_males=1,suscept_yrling_females=138, infected_yrling_females=0,suscept_adult_males=134,infected_adult_males=0,suscept_adult_females=413,infected_adult_females=0, suscept_fawn_males=229,infected_fawn_males=0,suscept_fawn_females=231,infected_fawn_females=0,groups=GROUPS, params=fits, xtrds=xtrends,ytrds= ytrends,cov_mats=cov_matrices) 
    
    month = 1
    day = 1  
    year = 1

    for s in range(1, timesteps+1):
       trial.move_deer()
       trial.incr_time_inf()
       trial.shed_prions()
       trial.ind_trans()
       trial.disease_mort()
       trial.fawn_mort()
           
       if not s % 30:
           trial.incr_age()
           trial.incr_birth_int()           
           trial.incr_disp_int() 
           trial.prion_decay()
        
           day=1
           month += 1
           if month > 12:
               year+=1
               month=1 
               day = 1
                
       if month == 1:
           trial.dir_trans_jan(dir_radius)
           trial.harvest_mort()
       if month == 2:
           trial.baseline_mort()
           trial.dir_trans_feb(dir_radius)
       if month == 3:
           trial.baseline_mort()
           trial.dir_trans_mar(dir_radius)
       if month == 4:
           trial.baseline_mort()
           trial.dir_trans_apr(dir_radius)
       if month == 5:
           trial.baseline_mort()
           trial.dir_trans_may(dir_radius)
           trial.birth()
       if month == 6:
           trial.baseline_mort()
           trial.spring_dispersal()
           trial.spring_emigration()
           trial.spring_immigration()
           trial.dir_trans_june(dir_radius)
           trial.birth()
       if month == 7:
           trial.baseline_mort()
           trial.dir_trans_july(dir_radius)     
       if month == 8:
           trial.baseline_mort()
           trial.dir_trans_aug(dir_radius)
       if month ==9:
           trial.baseline_mort()
           trial.dir_trans_sept(dir_radius)
       if month == 10:
           trial.fall_dispersal()
           trial.fall_emigration()
           trial.fall_immigration()
           trial.dir_trans_oct(dir_radius)
           trial.harvest_mort()
       if month == 11:
           trial.dir_trans_nov(dir_radius)
           trial.harvest_mort()
       if month ==12:
           trial.dir_trans_dec(dir_radius)
           trial.harvest_mort()  

       trial.list_comp()
       trial.male_comp()
       trial.female_comp()

       trial.increment_day()
       day += 1
            t_deer,e_deer,i_deer,prev,d_trans,i_trans,m_life,m_disease,d_mort,emigrant,m_prev,f_prev,prion_cells=trial.stats()
       T_total_deer[s-1]+=t_deer
       T_exposed_deer[s-1]+=e_deer
       T_infectious_deer[s-1]+=i_deer
       T_prevalence[s-1]+=prev
       T_dir_trans[s-1]+=d_trans
       T_ind_trans[s-1]+=i_trans
       T_max_life[s-1]+=m_life
       T_max_disease[s-1]+=m_disease
       T_dz_mort[s-1]+=d_mort
       T_emigration[s-1]+=emigrant
       T_m_prevalence[s-1]+=m_prev
       T_f_prevalence[s-1]+=f_prev
       T_shed_prions[s-1]+=prion_cells  
          
       data[0,:] = T_total_deer
       data[1,:] = T_exposed_deer
       data[2,:] = T_infectious_deer
       data[3,:] = T_prevalence
       data[4,:] = T_dir_trans
       data[5,:] = T_ind_trans
       data[6,:] = T_max_life
       data[7,:] = T_max_disease
       data[8,:] = T_dz_mort
       data[9,:] = T_emigration
       data[10,:] = T_m_prevalence
       data[11,:] = T_f_prevalence
       data[12,:] = T_shed_prions

    return data

res = disease(3600)   #total timesteps
np.savetxt(PATH+'/Total_deer/rank_'+str(rank)+'_total_deer',res[0,:],delimiter=',')
np.savetxt(PATH+'/Total_exp_deer/rank_'+str(rank)+'_exposed_deer',res[1,:],delimiter=',')
np.savetxt(PATH+'/Total_infect_deer/rank_'+str(rank)+'_infect_deer',res[2,:],delimiter=',')
np.savetxt(PATH+'/Prevalence/rank_'+str(rank)+'_prev',res[3,:],delimiter=',')
np.savetxt(PATH+'/Direct_trans/rank_'+str(rank)+'_dir_trans',res[4,:],delimiter=',')
np.savetxt(PATH+'/Indirect_trans/rank_'+str(rank)+'_ind_trans',res[5,:],delimiter=',')
np.savetxt(PATH+'/Max_life/rank_'+str(rank)+'_max_life',res[6,:],delimiter=',')
np.savetxt(PATH+'/Max_disease/rank_'+str(rank)+'_max_dz',res[7,:],delimiter=',')
np.savetxt(PATH+'/Disease_mort/rank_'+str(rank)+'_dz_mort',res[8,:],delimiter=',')
np.savetxt(PATH+'/Emigrants/rank_'+str(rank)+'_emigration',res[9,:],delimiter=',')
np.savetxt(PATH+'/Prevalence_male/rank_'+str(rank)+'_m_prev',res[10,:],delimiter=',')
np.savetxt(PATH+'/Prevalence_female/rank_'+str(rank)+'_f_prev',res[11,:],delimiter=',') 
np.savetxt(PATH+'/Prion_cells/rank_'+str(rank)+'_prion_cells',res[11,:],delimiter=',')